# Getting Started with PennyLane

In [ ]:
%pip install pennylane --upgrade
%pip install pennylane-qiskit --upgrade

## Quantum Circuits

In [1]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(y, wires=1)
    return qml.expval(qml.PauliZ(1))

In [3]:
shots_list = [5, 10, 1000]
dev = qml.device("default.qubit", wires=2, shots=shots_list)

In [4]:
circuit = qml.QNode(my_quantum_function, dev)
circuit(np.pi/2, np.pi/2)
print(qml.draw(circuit)(np.pi/2, np.pi/2))

0: ──RZ(1.57)─╭●───────────┤     
1: ───────────╰X──RY(1.57)─┤  <Z>


In [ ]:
qml.drawer.use_style("black_white")
fig, ax = qml.draw_mpl(circuit)(np.pi/2, np.pi/2)
plt.show()

In [ ]:
## QNode Decorator ##

dev = qml.device('default.qubit', wires=2, shots=10)

@qml.qnode(dev)
def circuit(x):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(x, wires=1)
    return qml.expval(qml.PauliZ(1))

result = circuit(np.pi/2)
result

## Loading and calling a parametrized Qiskit QuantumCircuit object while using the QNode decorator

In [26]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import numpy as np

dev = qml.device('default.qubit', wires=2, shots=10)

theta = Parameter('θ')

qc = QuantumCircuit(2)
qc.rz(theta, [0])
qc.rx(theta, [0])
qc.cx(0, 1)

@qml.qnode(dev)
def quantum_circuit_with_loaded_subcircuit(x):
    qml.from_qiskit(qc)({theta: x})
    return qml.expval(qml.PauliZ(0))

angle = np.pi/2
result = quantum_circuit_with_loaded_subcircuit(angle)
result

np.float64(-0.6)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    channel="ibm_quantum", token="afdb287966d333d556c9f84dab96673882fb3b31ae3de4133667f7450ce4905911d542708a216be1b4a8152cb901792907fc1fbca61ea31cc4144319b7c55983",
    set_as_default=True, overwrite=True, instance="usc/phys513/phys513p"
    )

# To access saved credentials for the IBM quantum channel and select an instance
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=10)

In [32]:
dev = qml.device('qiskit.remote', wires=4, backend=backend)

In [34]:
# Choose a backend from the available ones
backend_name = 'ibm_brisbane'
backend = service.backend(backend_name)  

# Define the Quantum Device
dev = qml.device(
    'qiskit.remote',           
    wires=4,
    backend=backend,
    shots=1024
)

In [35]:
dev = qml.device("default.qubit")

def do(k):

    qml.StatePrep([1,k], wires = [0], normalize = True)

def apply(theta):

    qml.IsingXX(theta, wires = [1,2])

@qml.qnode(dev)
def do_apply_undo(k,theta):
    """
    Applies V, controlled-U, and the inverse of V
    Args: 
    - k, theta: The parameters for do and apply (V and U) respectively
    Returns:
    - (np.array): The output state of the routine
    """

    do(k)
    qml.ctrl(apply, control = 0)(theta)
    qml.adjoint(do)(k)

    return qml.state()

k, theta = 0.5, 0.8

print("The output state is: \n", do_apply_undo(k, theta))

The output state is: 
 [ 0.9842122+0.j          0.       +0.j          0.       +0.j
  0.       -0.07788367j -0.0315756+0.j          0.       +0.j
  0.       +0.j          0.       -0.15576734j]


In [36]:
circuit = qml.QNode(do_apply_undo, dev)
print(qml.draw(circuit)(0.5, 0.8))

0: ──|Ψ⟩─╭●──────────────|Ψ⟩†─┤  State
1: ──────├IsingXX(0.80)───────┤  State
2: ──────╰IsingXX(0.80)───────┤  State
